In [1]:
from __future__ import print_function
from __future__ import division

In [2]:
import torch
from torch.utils.data import DataLoader

In [3]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

In [4]:
import torchvision
from torchvision import transforms

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import time
import os
import copy
from tqdm.notebook import tqdm

In [7]:
from lightning_model import init_cls_model

In [8]:
from dataset_loaders import IsRestaurantDataset

ImportError: cannot import name 'IsRestaurantDataset'

In [11]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print("PyTorch Lightning Version: ",pl.__version__)

PyTorch Version:  1.8.1+cu102
Torchvision Version:  0.9.1+cu102
PyTorch Lightning Version:  1.1.8


In [12]:
os.chdir('yelp_academic')

FileNotFoundError: [Errno 2] No such file or directory: 'yelp_academic'

In [13]:
photo_dir = 'photos/'
csv_path = "data/business_restaurant.csv"

In [20]:
FLAGS = {
    'model_name': 'custom',
    'num_classes': 2,
    'batch_size': 128,
    'num_workers': 0,
    'learning_rate': 0.02,
    'momentum': 0.9,
    'num_epochs': 3,
    'output_dim': 6,
    'class_weight': [3.513, 1.63, 6.084, 9.839, 6.502, 4.625],
    'multilabel': True,
    'feature_extract': True,
    'use_pretrained': True
        }

In [21]:
transfer_model, input_size = init_cls_model(FLAGS)

In [22]:
transfer_model

LightningTransfer(
  (model): CustomCNN(
    (conv1): Sequential(
      (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
      (1): MaxPool2d(kernel_size=4, stride=3, padding=0, dilation=1, ceil_mode=False)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): Dropout2d(p=0.5, inplace=False)
      (3): ReLU()
    )
    (conv3): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
      (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): Dropout2d(p=0.5, inplace=False)
      (3): ReLU()
    )
    (fc1): Linear(in_features=8192, out_features=4096, bias=True)
    (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    (fc3): Linear(in_features=1024, out_features=6, bias=True)
    (drop): Dropout(p=0.5, inplace=False)
    (relu): ReLU()
  )
  (criterion): BCEWithLogits

In [14]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'dev': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [15]:
print("Initializing Datasets and Dataloaders...")

Initializing Datasets and Dataloaders...


In [16]:
csv_file = pd.read_csv(csv_path)

In [17]:
csv_file.head()

,photo_id,is_business
0,3V7tgMx3Qw5L9ZjRLNbthA,True
1,fZo1owoYqwAHW7uZlTz1XQ,False
2,zwOCQ8w3gFuF3zi_dyIWpw,False
3,hQBfeDngFMpB9HX2CPKtag,True
4,Fjh4N5B38vJWVbuQk-v3aQ,True


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train, test = train_test_split(csv_file, test_size=0.2, random_state=42, stratify=csv_file.is_business)

In [20]:
dev, test = train_test_split(test, test_size=0.5, random_state=42, stratify=test.is_business)

In [21]:
data_frames = {'train': train,
              'dev': dev,
              'test': test}

In [22]:
# Create training and validation datasets
image_datasets = {x: IsRestaurantDataset(data_frames[x], photo_dir, data_transforms[x]) for x in ['train', 'dev', 'test']}
# Create training and validation dataloaders
dataloaders_dict = {x: DataLoader(image_datasets[x],
                                  batch_size=FLAGS['batch_size'],
                                  shuffle=True,
                                  num_workers=FLAGS['num_workers']) for x in ['train', 'dev']}
dataloaders_dict['test'] = DataLoader(image_datasets['test'],
                                      batch_size=FLAGS['batch_size'],
                                      shuffle=False,
                                      num_workers=FLAGS['num_workers'])

In [23]:
dataloaders_dict

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f02b17140b8>,
 'dev': <torch.utils.data.dataloader.DataLoader at 0x7f02b1714208>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f02b16fbfd0>}

In [24]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
early_stop_callback = EarlyStopping(monitor='val_loss', patience=2)

In [25]:
trainer = pl.Trainer(callbacks=[checkpoint_callback, early_stop_callback])

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [26]:
trainer.fit(transfer_model, dataloaders_dict['train'], dataloaders_dict['dev'])


  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
1.0 K     Trainable params
11.2 M    Non-trainable params
11.2 M    Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

1

In [ ]:
trainer.test(transfer_model, dataloaders_dict['test'])